In [1]:
# python libraries
import numpy as np
from IPython.display import HTML, display
# pydrake imports
from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder, LogVectorOutput, namedview,
                         MeshcatVisualizer, ModelVisualizer, Parser, Simulator, StartMeshcat,
                         MathematicalProgram, eq, SnoptSolver, PiecewisePolynomial, TrajectorySource,
                         MultibodyPlant, SceneGraph, MultibodyPositionToGeometryPose, PlanarSceneGraphVisualizer, inv)

from pydrake import multibody

from underactuated import ConfigureParser
from underactuated.scenarios import AddFloatingRpyJoint

# URDF

In [2]:
bar1_link = """
  <link name="bar1">

    <inertial>
      <origin xyz="0.75 0 2.5" />
      <mass value="0" />
    </inertial>

    <visual>
      <origin xyz="0.75 0 2.5" />
      <geometry>
        <box size=".1 .1 .1" />
      </geometry>
      <material>
        <color rgba="0 1 0 1" />
      </material>
    </visual>

  </link>
"""
bar2_link = """
  <link name="bar2">

    <inertial>
      <origin xyz="-0.75 0 1.7" />
      <mass value="0" />
    </inertial>

    <visual>
      <origin xyz="-0.75 0 1.7" />
      <geometry>
        <box size=".1 .1 .1" />
      </geometry>
      <material>
        <color rgba="0 1 0 1" />
      </material>
    </visual>

  </link>
"""
hook_link = """
  <link name="hook">

    <inertial>
      <origin xyz="0.75 0 2.5" />
      <mass value="0.1" />
      <inertia ixx="0.1" ixy="0" ixz="0" iyy="0.1" iyz="0" izz="0.1" />
    </inertial>

    <visual>
      <origin xyz="0.75 0 2.5" />
      <geometry>
        <box size=".1 .1 .1" />
      </geometry>
      <material>
        <color rgba="0 0 1 1" />
      </material>
    </visual>

  </link>
"""
pendulum0_link = """ 
  <link name="pendulum0">

    <inertial>
      <origin xyz="0 0 -0.4" rpy="0 0 0" />
      <mass value="1" />
      <inertia ixx="0.05333" ixy="0" ixz="0" iyy="0.05333" iyz="0" izz="0" />
    </inertial>
    
    <visual>
      <origin xyz="0 0 -0.4" rpy="0 0 0" />
      <geometry>
        <cylinder length="0.8" radius=".05" />
      </geometry>
      <material>
        <color rgba="1 0 0 1" />
      </material>
    </visual>
  </link>
"""
pendulum1_link = """ 
  <link name="pendulum1">

    <inertial>
      <origin xyz="0 0 -0.4" rpy="0 0 0" />
      <mass value="1" />
      <inertia ixx="1" ixy="0" ixz="0" iyy="0.053333" iyz="0" izz="1"/>
    </inertial>
    
    <visual>
      <origin xyz="0 0 -0.4" rpy="0 0 0" />
      <geometry>
        <cylinder length="0.8" radius=".05" />
      </geometry>
      <material>
        <color rgba="1 0 0 1" />
      </material>
    </visual>
  </link>
"""

In [3]:
bar1_weld = """
  <joint name="bar1_weld" type="fixed">
    <parent link="world"/>
    <child link="bar1"/>
  </joint>
"""
bar2_weld = """
  <joint name="bar2_weld" type="fixed">
    <parent link="world"/>
    <child link="bar2"/>
  </joint>
"""
pendulum0_joint = """
  <joint name="theta0" type="continuous">
    <parent link="hook" />
    <child link="pendulum0" />
    <origin xyz="0.75 0 2.5" />
    <axis xyz="0 -1 0" />
  </joint>
"""
pendulum1_joint = """
  <joint name="theta1" type="continuous">
    <parent link="pendulum0" />
    <child link="pendulum1" />
    <origin xyz="0 0 -0.8" />
    <axis xyz="0 -1 0" />
  </joint>
"""
transmission = """
  <transmission type="SimpleTransmission" name="ab_force">
    <actuator name="force" />
    <joint name="theta1" />
  </transmission>
"""

In [4]:
robot_urdf=f"""
<?xml version="1.0"?><robot name="robot">
{bar1_link}
{bar2_link}
{bar1_weld}
{bar2_weld}
{hook_link}
{pendulum0_link}
{pendulum0_joint}
{pendulum1_link}
{pendulum1_joint}
{transmission}
  </robot>
</xml>
"""

# Creating the MultibodyPlant

The default floating joint is replaced by an RPY joint to avoid quaternion algebra and have the velocities truly be the derivatives of the states.

In [5]:
builder=DiagramBuilder()
robot,scene_graph=AddMultibodyPlantSceneGraph(builder,time_step=0.001)
(model_instance,)=Parser(robot).AddModelsFromString(robot_urdf,"urdf")
AddFloatingRpyJoint(robot,robot.GetFrameByName("hook"),model_instance,use_ball_rpy=False)
robot.Finalize()
robot=robot.ToAutoDiffXd()
robot.set_name("robot")
state=namedview("States",robot.GetStateNames(robot.GetModelInstanceByName("robot")))
print(state._fields)

diagram=builder.Build()

('x_x', 'y_x', 'z_x', 'rz_q', 'ry_q', 'rx_q', 'theta0_q', 'theta1_q', 'x_v', 'y_v', 'z_v', 'rz_w', 'ry_w', 'rx_w', 'theta0_w', 'theta1_w')


# Defining the Manipulator Equations

When the robot is on a bar, the terms corresponding to the floating joint are discarded, and qdd is constrained to 0 instead.

In [6]:
nq=8
def manipulator_equations(mode):
    def helper(vars):
        context=robot.CreateDefaultContext()
        split_at=[nq,2*nq,3*nq]
        q,qd,qdd,u=np.split(vars,split_at)
        robot.SetPositions(context,q)
        robot.SetVelocities(context,qd)
        M=robot.CalcMassMatrixViaInverseDynamics(context)
        Cv=robot.CalcBiasTerm(context)
        tauG=robot.CalcGravityGeneralizedForces(context)
        B=robot.MakeActuationMatrix()
        err=M.dot(qdd)+Cv-tauG-B.dot(u)
        if mode=="on_bar":
            return np.concatenate((qdd[:nq-2],err[nq-2:nq]))
        else:
            return err
    return helper

# tests
print(manipulator_equations("in_air")([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-9.81,0,0,0,0,0,0]))
print(manipulator_equations("on_bar")([0]*25))

[<AutoDiffXd 0.0 nderiv=0> <AutoDiffXd 0.0 nderiv=0>
 <AutoDiffXd 0.0 nderiv=0> <AutoDiffXd 0.0 nderiv=0>
 <AutoDiffXd 1.7763568394002505e-15 nderiv=0> <AutoDiffXd 0.0 nderiv=0>
 <AutoDiffXd 0.0 nderiv=0> <AutoDiffXd 0.0 nderiv=0>]
[0 0 0 0 0 0 <AutoDiffXd 0.0 nderiv=0> <AutoDiffXd 0.0 nderiv=0>]


# Setting Up the MathematicalProgram

In [7]:
T=100
h_min=0.005
h_max=0.05
prog=MathematicalProgram()
q=prog.NewContinuousVariables(rows=T+1,cols=nq,name="q")
qd=prog.NewContinuousVariables(rows=T+1,cols=nq,name="qd")
qdd=prog.NewContinuousVariables(rows=T,cols=nq,name="qdd")
h=prog.NewContinuousVariables(T,name="h")
u=prog.NewContinuousVariables(T,name="u")
prog.AddBoundingBoxConstraint([h_min]*T,[h_max]*T,h)
prog.AddConstraint(eq(q[0],[0]*nq))
prog.AddConstraint(eq(qd[0],[0]*nq))
for t in range(T):
    prog.AddConstraint(eq(q[t+1],q[t]+h[t]*qd[t]))
    prog.AddConstraint(eq(qd[t+1],qd[t]+h[t]*qdd[t]))
for t in range(T):
    vars=np.concatenate((q[t+1],qd[t+1],qdd[t],[u[t]]))
    prog.AddConstraint(manipulator_equations("on_bar" if t<T/2 else "in_air"),lb=[0]*nq,ub=[0]*nq,vars=vars)
prog.AddConstraint(eq(q[-1][:3],[-1.5,0,-0.8]))
prog.AddCost(sum([u[i]**2 for i in range(T)]))

In [8]:
solver=SnoptSolver()
tol=1e-3
prog.SetSolverOption(solver.solver_id(),"Feasibility tolerance",tol)
prog.SetSolverOption(solver.solver_id(),"Major feasibility tolerance",tol)
prog.SetSolverOption(solver.solver_id(),"Major optimality tolerance",tol)
prog.SetSolverOption(solver.solver_id(),"Minor feasibility tolerance",tol)
prog.SetSolverOption(solver.solver_id(),"Minor optimality tolerance",tol)
result=solver.Solve(prog)
print(result.is_success())

True


In [13]:
q_opt=result.GetSolution(q)
qd_opt=result.GetSolution(qd)
h_opt=result.GetSolution(h)
x_opt=np.hstack((q_opt,qd_opt))

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.         -0.3222247
   1.03111747]
 ...
 [-1.47396432  0.         -0.74894987 ...  0.          3.5524985
  -3.56563154]
 [-1.48706272  0.         -0.77431651 ...  0.          3.54336468
  -3.56563154]
 [-1.5         0.         -0.8        ...  0.          3.53449183
  -3.56563154]]


# Animating

The animation is extended with the robot dangling on the second bar for dramatic effect.

In [22]:
simulator=Simulator(diagram)
simulator.set_publish_every_time_step(False)
simulator.get_mutable_integrator().set_fixed_step_mode(True)

context=simulator.get_mutable_context()
context.SetTime(0.)
context.SetDiscreteState(x_opt[-1])
robot=diagram.GetSystems()[0]
robot.get_actuation_input_port().FixValue(robot.GetMyContextFromRoot(context),[0])
robot_context=robot.GetMyContextFromRoot(context)
for i in range(4,10):
    joint=robot.get_joint(multibody.tree.JointIndex(i))
    joint.Lock(robot_context)

simulator.Initialize()
for t in range(200):
    simulator.AdvanceTo((t+1)*0.01)
    x_opt=np.concatenate((x_opt,context.get_discrete_state_vector().value().reshape((1,16))))

In [23]:
time_breaks_opt=np.array([sum(h_opt[:t]) for t in range(T+1)])
for t in range(200):
    time_breaks_opt=np.append(time_breaks_opt,time_breaks_opt[-1]+0.01)
x_opt_poly=PiecewisePolynomial.FirstOrderHold(time_breaks_opt,x_opt.T)

sim_robot=MultibodyPlant(time_step=0)
sim_scene_graph=SceneGraph()
sim_robot.RegisterAsSourceForSceneGraph(sim_scene_graph)
parser=Parser(sim_robot)
ConfigureParser(parser)
parser.AddModelsFromString(robot_urdf,"urdf")
AddFloatingRpyJoint(sim_robot,sim_robot.GetFrameByName("hook"),model_instance,use_ball_rpy=False)
sim_robot.Finalize()

sim_builder=DiagramBuilder()
source=sim_builder.AddSystem(TrajectorySource(x_opt_poly))
sim_builder.AddSystem(sim_scene_graph)
pos_to_pose=sim_builder.AddSystem(MultibodyPositionToGeometryPose(sim_robot,input_multibody_state=True))
sim_builder.Connect(source.get_output_port(0),pos_to_pose.get_input_port())
sim_builder.Connect(pos_to_pose.get_output_port(),sim_scene_graph.get_source_pose_port(sim_robot.get_source_id()),)

xlim=[-3,2]
ylim=[0,3]
visualizer=sim_builder.AddSystem(PlanarSceneGraphVisualizer(sim_scene_graph,xlim=xlim,ylim=ylim,show=False))
sim_builder.Connect(sim_scene_graph.get_query_output_port(),visualizer.get_input_port(0))
simulator=Simulator(sim_builder.Build())

visualizer.start_recording()
simulator.AdvanceTo(x_opt_poly.end_time())
ani = visualizer.get_recording_as_animation()
HTML(ani.to_jshtml())

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c8b2dc05-5a36-4f34-a5f6-f2e8ed44ad6d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>